In [1]:
#imports
from qiskit import IBMQ, Aer, assemble, transpile, QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.visualization import array_to_latex
from qiskit.tools.monitor import job_monitor 
from qiskit.providers.ibmq import least_busy
from qiskit import *
from qiskit.visualization import *


from qiskit.providers.aer import QasmSimulator
from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error

import numpy as np
from numpy import sum
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from math import pi

/tmp/ipykernel_59/1153469991.py:11: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter


In [2]:
#constructing qiskit's calibration matrix
qc = QuantumCircuit(3,3)
qc.h(0)
qc.cx(0,1)
qc.cx(0,2)  
qc.measure([0, 1,2], [0, 1,2])

qr = QuantumRegister(3)
meas_calibs, state_labels = complete_meas_cal(qr=qr, circlabel='mcal')

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-education', group='fermilab-1', project='qjs-for-hep')
backend = provider.get_backend('ibm_lagos')
t_qc = transpile(meas_calibs, backend, optimization_level=3)
job = backend.run(t_qc, shots=8000)
job_monitor(job, interval=2)

meas_fitter = CompleteMeasFitter(job.result(), state_labels, circlabel='mcal')
qiskitcalm= meas_fitter.cal_matrix

Job Status: job has successfully run


In [2]:
qiskitcalm= [[9.69500e-01, 1.27500e-02, 1.36250e-02, 2.50000e-04, 6.62500e-03, 0.00000e+00, 0.00000e+00, 0.00000e+00,],[1.13750e-02, 9.72000e-01, 1.25000e-04, 1.40000e-02, 0.00000e+00, 8.75000e-03, 0.00000e+00, 0.00000e+00],[1.08750e-02, 1.25000e-04, 9.72500e-01, 1.02500e-02, 6.25000e-04, 0.00000e+00, 6.50000e-03, 1.25000e-04], [0.00000e+00, 1.06250e-02, 9.12500e-03, 9.71500e-01, 0.00000e+00, 5.00000e-04, 0.00000e+00, 7.62500e-03], [8.12500e-03, 0.00000e+00, 1.25000e-04, 0.00000e+00, 9.72625e-01, 1.31250e-02, 1.36250e-02, 2.50000e-04], [1.25000e-04, 4.50000e-03, 0.00000e+00, 0.00000e+00, 1.00000e-02, 9.68625e-01, 2.50000e-04, 1.28750e-02], [0.00000e+00, 0.00000e+00, 4.50000e-03, 1.25000e-04, 1.01250e-02, 0.00000e+00, 9.67250e-01, 1.15000e-02], [0.00000e+00, 0.00000e+00, 0.00000e+00, 3.87500e-03, 0.00000e+00, 9.00000e-03, 1.23750e-02, 9.67625e-01]]
print(qiskitcalm)

[[0.9695, 0.01275, 0.013625, 0.00025, 0.006625, 0.0, 0.0, 0.0], [0.011375, 0.972, 0.000125, 0.014, 0.0, 0.00875, 0.0, 0.0], [0.010875, 0.000125, 0.9725, 0.01025, 0.000625, 0.0, 0.0065, 0.000125], [0.0, 0.010625, 0.009125, 0.9715, 0.0, 0.0005, 0.0, 0.007625], [0.008125, 0.0, 0.000125, 0.0, 0.972625, 0.013125, 0.013625, 0.00025], [0.000125, 0.0045, 0.0, 0.0, 0.01, 0.968625, 0.00025, 0.012875], [0.0, 0.0, 0.0045, 0.000125, 0.010125, 0.0, 0.96725, 0.0115], [0.0, 0.0, 0.0, 0.003875, 0.0, 0.009, 0.012375, 0.967625]]


In [3]:
print("Qiskit's calibration matrix:")
array_to_latex(qiskitcalm)

Qiskit's calibration matrix:


<IPython.core.display.Latex object>

In [5]:
#sim code clement sent (thanks clement)
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-education', group='fermilab-1', project='qjs-for-hep')
provider.backends()

N = 2
epsilon = 0.5
mass = 1.0

def gauge_kinetic(epsilon):
    circuit=QuantumCircuit(1)
    circuit.rx(-epsilon/2,0)
    U_kg = circuit.to_gate()
    U_kg.name = "U$_{Kg}$"
    return U_kg

def fermion_mass(epsilon,mass,eta):
    circuit=QuantumCircuit(1)
    circuit.rz(-epsilon*mass * eta,0)
    U_m = circuit.to_gate()
    U_m.name = "U$_m$"
    return U_m

def fermion_hopping_opt2(epsilon,eta):
    circuit= QuantumCircuit(3)
    circuit.cx(0,2)
    circuit.h(0)
    circuit.cx(1,0)
    circuit.cx(0,2)
    circuit.rz(epsilon/4 * eta,0)
    circuit.rz(-epsilon/4 * eta,2)
    circuit.cx(0,2)
    circuit.cx(1,0)
    circuit.h(0)
    circuit.cx(0,2)
    U_fho2 = circuit.to_gate()
    U_fho2.name = "U$_{fho2}$"
    return U_fho2

def get_mean_fermion_number(counts):
    mean = 0
    total_counts = sum(counts.values())
    for s in counts:
        p = s[-1]
        if p == '1':
            mean += 1./total_counts * counts[s]
    return mean

def get_bootstrap_error(counts):
    nshots=sum(counts.values())
    B = 100
    k = list(counts.keys())
    prob = [counts[a]/nshots for a in k]
    means = []
    for b in range(B):
        m = 0
        samples = numpy.random.choice(k, size=nshots, p=prob)
        for s in samples:
            p = s[-1]
            if p == '1':
                m += 1./nshots
        means.append(m)
    return numpy.std(means)

counts=[]
sim_counts=[]
means=[]
errs=[]
sim_means=[]
sim_errs=[]
Ts=[]

for T in range(int(0/epsilon),int(3/epsilon)):
    Ts.append(T)
    qc = QuantumCircuit(2*N-1, 2*N-1)

    qc.x(0)
    qc.h(0)

    for t in range(T):
        for n in range(0,2*N,2):
            qc.append(fermion_mass(epsilon,mass,(-1)**(n/2+1)),[n])
        for l in range(1,2*N-1,2):
            qc.append(gauge_kinetic(epsilon),[l])
        for n in range(0,2*N-3,2):
            qc.append(fermion_hopping_opt2(epsilon, (-1)**(n/2)),[n,n+1,n+2])
        
    qc.measure(range(3), range(3))
    
    sim_backend = Aer.get_backend('qasm_simulator')
    backend = provider.get_backend('ibm_lagos')
    
    sim_tc = transpile(qc, sim_backend, optimization_level=3)
    tc = transpile(qc, backend, optimization_level=3)

    sim_job = sim_backend.run(sim_tc,shots=4096) 
    job = backend.run(tc)

    sim_result= sim_job.result()
    sim_counts.append(sim_result.get_counts())

    results= job.result().get_counts()
    counts.append(results)

ibmqfactory.load_account:WARNING:2022-02-04 02:33:53,112: Credentials are already in use. The existing account in the session will be replaced.


In [7]:
counts= [{'000': 1875, '001': 2070, '010': 18, '011': 19, '100': 10, '101': 8}, {'000': 1951, '001': 1648, '010': 51, '011': 25, '100': 207, '101': 115, '110': 2, '111': 1}, {'000': 1644, '001': 1550, '010': 235, '011': 129, '100': 226, '101': 173, '110': 21, '111': 22}, {'000': 1405, '001': 1544, '010': 346, '011': 235, '100': 209, '101': 174, '110': 64, '111': 23}, {'000': 1227, '001': 1249, '010': 572, '011': 217, '100': 215, '101': 353, '110': 92, '111': 75}, {'000': 1335, '001': 1287, '010': 388, '011': 176, '100': 251, '101': 403, '110': 91, '111': 69}]
print(counts)

[{'000': 1875, '001': 2070, '010': 18, '011': 19, '100': 10, '101': 8}, {'000': 1951, '001': 1648, '010': 51, '011': 25, '100': 207, '101': 115, '110': 2, '111': 1}, {'000': 1644, '001': 1550, '010': 235, '011': 129, '100': 226, '101': 173, '110': 21, '111': 22}, {'000': 1405, '001': 1544, '010': 346, '011': 235, '100': 209, '101': 174, '110': 64, '111': 23}, {'000': 1227, '001': 1249, '010': 572, '011': 217, '100': 215, '101': 353, '110': 92, '111': 75}, {'000': 1335, '001': 1287, '010': 388, '011': 176, '100': 251, '101': 403, '110': 91, '111': 69}]


In [10]:
# my own code to get the results into a state i can actually do something with. not sure if this is right or not. 
bigmatrix = []
for dic in counts:
    keys= dic.keys()
    values = dic.values()
    numoftimes= []
    state= []
    for value in values:
        numoftimes.append(value/4096)
    for key in keys:
        state.append(key)

    while len(state) < 8:
        state.append(0)

    if state != ['000','001','010','011','100','101','110','111']:
        if state[0] != '000':
            state.insert(0,'000')
            numoftimes.insert(0,0)
        if state[1] != '001':
            state.insert(1,'001')
            numoftimes.insert(1,0)
        if state[2] != '010':
            state.insert(2,'010')
            numoftimes.insert(2,0)
        if state[3] != '011':
            state.insert(3,'011')
            numoftimes.insert(3,0)
        if state[4] != '100':
            state.insert(4,'100')
            numoftimes.insert(4,0)
        if state[5] != '101':
            state.insert(5,'101')
            numoftimes.insert(5,0)
        if state[6] != '110':
            state.insert(6,'110')
            numoftimes.insert(6,0)
        if state[7] != '111':
            state.insert(7,'111')
            numoftimes.insert(7,0)
    bigmatrix.append(numoftimes)
print(bigmatrix)

[[0.457763671875, 0.50537109375, 0.00439453125, 0.004638671875, 0.00244140625, 0.001953125, 0, 0], [0.476318359375, 0.40234375, 0.012451171875, 0.006103515625, 0.050537109375, 0.028076171875, 0.00048828125, 0.000244140625], [0.4013671875, 0.37841796875, 0.057373046875, 0.031494140625, 0.05517578125, 0.042236328125, 0.005126953125, 0.00537109375], [0.343017578125, 0.376953125, 0.08447265625, 0.057373046875, 0.051025390625, 0.04248046875, 0.015625, 0.005615234375], [0.299560546875, 0.304931640625, 0.1396484375, 0.052978515625, 0.052490234375, 0.086181640625, 0.0224609375, 0.018310546875], [0.325927734375, 0.314208984375, 0.0947265625, 0.04296875, 0.061279296875, 0.098388671875, 0.022216796875, 0.016845703125]]


In [15]:
bigmatrix= [[0.457763671875, 0.50537109375, 0.00439453125, 0.004638671875, 0.00244140625, 0.001953125, 0, 0], [0.476318359375, 0.40234375, 0.012451171875, 0.006103515625, 0.050537109375, 0.028076171875, 0.00048828125, 0.000244140625], [0.4013671875, 0.37841796875, 0.057373046875, 0.031494140625, 0.05517578125, 0.042236328125, 0.005126953125, 0.00537109375], [0.343017578125, 0.376953125, 0.08447265625, 0.057373046875, 0.051025390625, 0.04248046875, 0.015625, 0.005615234375], [0.299560546875, 0.304931640625, 0.1396484375, 0.052978515625, 0.052490234375, 0.086181640625, 0.0224609375, 0.018310546875], [0.325927734375, 0.314208984375, 0.0947265625, 0.04296875, 0.061279296875, 0.098388671875, 0.022216796875, 0.016845703125]]

In [20]:
res0=bigmatrix[0]
res0 = np.array([res0]).T
res1=bigmatrix[1]
res1 = np.array([res1]).T
res2=bigmatrix[2]
res2 = np.array([res2]).T
res3=bigmatrix[3]
res3 = np.array([res3]).T
res4=bigmatrix[4]
res4 = np.array([res4]).T
res5=bigmatrix[5]
res5 = np.array([res5]).T

array_to_latex(res0)

<IPython.core.display.Latex object>

In [21]:
array_to_latex(res1)

<IPython.core.display.Latex object>

In [22]:
array_to_latex(res2)

<IPython.core.display.Latex object>

In [23]:
array_to_latex(res3)

<IPython.core.display.Latex object>

In [24]:
array_to_latex(res4)

<IPython.core.display.Latex object>

In [25]:
array_to_latex(res5)

<IPython.core.display.Latex object>

In [28]:
#I think I'm doing the math right here?
qiskitcalminv= np.linalg.inv(qiskitcalm)

qiskiterrorcorrected0= np.matmul(qiskitcalminv, res0)
array_to_latex(qiskiterrorcorrected0)

<IPython.core.display.Latex object>

In [29]:
qiskiterrorcorrected1= np.matmul(qiskitcalminv, res1)
array_to_latex(qiskiterrorcorrected1)

<IPython.core.display.Latex object>

In [30]:
qiskiterrorcorrected2= np.matmul(qiskitcalminv, res2)
array_to_latex(qiskiterrorcorrected2)

<IPython.core.display.Latex object>

In [31]:
qiskiterrorcorrected3= np.matmul(qiskitcalminv, res3)
array_to_latex(qiskiterrorcorrected3)

<IPython.core.display.Latex object>

In [32]:
qiskiterrorcorrected4= np.matmul(qiskitcalminv, res4)
array_to_latex(qiskiterrorcorrected4)

<IPython.core.display.Latex object>

In [33]:
qiskiterrorcorrected5= np.matmul(qiskitcalminv, res5)
array_to_latex(qiskiterrorcorrected5)

<IPython.core.display.Latex object>